# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных:

* id — уникальный идентификатор скважины;
* f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
* product — объём запасов в скважине (тыс. баррелей).

## Загрузка и подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [2]:
data_1 = pd.read_csv("/datasets/geo_data_0.csv")
data_2 = pd.read_csv("/datasets/geo_data_1.csv")
data_3 = pd.read_csv("/datasets/geo_data_2.csv")

In [3]:
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [7]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [8]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Дубликаты

In [9]:
print("Количество дубликатов в data_1:",data_1.duplicated().sum())
print("Количество дубликатов в data_2:",data_2.duplicated().sum())
print("Количество дубликатов в data_3:",data_3.duplicated().sum())

Количество дубликатов в data_1: 0
Количество дубликатов в data_2: 0
Количество дубликатов в data_3: 0


In [10]:
data_1[data_1["product"] == 0]['product'].count()

1

In [11]:
data_2[data_2["product"] == 0]['product'].count()

8235

In [12]:
data_3[data_3["product"] == 0]['product'].count()

1

Признак `id` не нужен в дальнейшем анализе. Поэтому можно удлать данный признак.

In [13]:
data_1.drop(["id"], axis=1, inplace=True)

In [14]:
data_2.drop(["id"], axis=1, inplace=True)

In [15]:
data_3.drop(["id"], axis=1, inplace=True)

In [16]:
data_1.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


## Обучение и проверка модели

### Разделение на выборки

In [18]:
def split_data(data_split):
    features = data_split.drop("product",axis=1)
    target = data_split["product"]
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, 
                                                                                  test_size=0.25, random_state=12345)
    numeric = ['f0', 'f1', 'f2']
    scaler = StandardScaler()
    scaler.fit(features_train[numeric])

    features_train[numeric] = scaler.transform(features_train[numeric])
    features_valid[numeric] = scaler.transform(features_valid[numeric])
    
    return features_train, features_valid, target_train, target_valid

In [19]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = split_data(data_1)

In [20]:
print(f'Размер исходного датафрейма: {data_1.shape}')
print(f'Размер признака обучающей выборке: {features_train_1.shape}')
print(f'Размер целевого признака обучающей выборке: {target_train_1.shape[0]}')
print(f'Размер признака валидационной выборки: {features_valid_1.shape}')
print(f'Размер целевого признака валидационной выборки: {target_valid_1.shape[0]}')

Размер исходного датафрейма: (100000, 4)
Размер признака обучающей выборке: (75000, 3)
Размер целевого признака обучающей выборке: 75000
Размер признака валидационной выборки: (25000, 3)
Размер целевого признака валидационной выборки: 25000


In [21]:
features_train_2, features_valid_2, target_train_2, target_valid_2 = split_data(data_2)

In [22]:
print(f'Размер исходного датафрейма: {data_2.shape}')
print(f'Размер признака обучающей выборке: {features_train_2.shape}')
print(f'Размер целевого признака обучающей выборке: {target_train_2.shape[0]}')
print(f'Размер признака валидационной выборки: {features_valid_2.shape}')
print(f'Размер целевого признака валидационной выборки: {target_valid_2.shape[0]}')

Размер исходного датафрейма: (100000, 4)
Размер признака обучающей выборке: (75000, 3)
Размер целевого признака обучающей выборке: 75000
Размер признака валидационной выборки: (25000, 3)
Размер целевого признака валидационной выборки: 25000


In [23]:
features_train_3, features_valid_3, target_train_3, target_valid_3 = split_data(data_3)

In [24]:
print(f'Размер исходного датафрейма: {data_3.shape}')
print(f'Размер признака обучающей выборке: {features_train_3.shape}')
print(f'Размер целевого признака обучающей выборке: {target_train_3.shape[0]}')
print(f'Размер признака валидационной выборки: {features_valid_3.shape}')
print(f'Размер целевого признака валидационной выборки: {target_valid_3.shape[0]}')

Размер исходного датафрейма: (100000, 4)
Размер признака обучающей выборке: (75000, 3)
Размер целевого признака обучающей выборке: 75000
Размер признака валидационной выборки: (25000, 3)
Размер целевого признака валидационной выборки: 25000


### Обучение модели

In [25]:
def model_predict(features_train, features_valid, target_train, target_valid):
    model = LinearRegression()
    model.fit(features_train,target_train)
    predictions_valid = pd.Series(model.predict(features_valid))
    rmse = mean_squared_error(target_valid, predictions_valid)**0.5
    srednee = predictions_valid.mean()
    
    return predictions_valid, rmse, srednee

### Регион 1

In [26]:
predictions_valid_1, rmse_1, srednee_1 = model_predict(features_train_1, 
                                                       features_valid_1, 
                                                       target_train_1, 
                                                       target_valid_1
                                                      )

In [27]:
print("RMSE модели:",rmse_1)
print("Средний запас сырья в регионе:",srednee_1)

RMSE модели: 37.5794217150813
Средний запас сырья в регионе: 92.59256778438035


### Регион 2

In [28]:
predictions_valid_2, rmse_2, srednee_2 = model_predict(features_train_2, 
                                                       features_valid_2, 
                                                       target_train_2, 
                                                       target_valid_2
                                                      )

In [29]:
print("RMSE модели:",rmse_2)
print("Средний запас сырья в регионе:",srednee_2)

RMSE модели: 0.893099286775617
Средний запас сырья в регионе: 68.728546895446


### Регион 3

In [30]:
predictions_valid_3, rmse_3, srednee_3 = model_predict(features_train_3, 
                                                       features_valid_3, 
                                                       target_train_3, 
                                                       target_valid_3
                                                      )

In [31]:
print("RMSE модели:",rmse_3)
print("Средний запас сырья в регионе:",srednee_3)

RMSE модели: 40.02970873393434
Средний запас сырья в регионе: 94.96504596800489


### Вывод:
Исходя из прогноза модели, то средний запас у 1 и 3 региона намного больше, чем у второго. Оданко, у данных регионов RMSE модели тоже значительно выше, чем у второго региона. Это говорит о том, что во втором регионе наибольшая точность предсказания сырья

## Подготовка к расчёту прибыли

### Константы

In [32]:
explore = 500
mining = 200 
budget = 10000000000
revenue_per_barrel = 450000
loss_probability = 0.025
bootstrap_number = 1000

### Расчет минимального объема сырья

In [33]:
print("Объем сырья для безубыточной разработки 1 скважины:", budget / mining / revenue_per_barrel)

Объем сырья для безубыточной разработки 1 скважины: 111.11111111111111


In [34]:
print("Средний запас сырья в регионе 1:",srednee_1)
print("Средний запас сырья в регионе 2:",srednee_2)
print("Средний запас сырья в регионе 3:",srednee_3)

Средний запас сырья в регионе 1: 92.59256778438035
Средний запас сырья в регионе 2: 68.728546895446
Средний запас сырья в регионе 3: 94.96504596800489


#### Вывод:
Таким образом, для безубыточной работы необходимо порядка 111 тыч. баррелей. А средние значения регионов ниже данного порога, что говорит о возможности понести убытки при добыче

### Функция расчета прибыли

In [35]:
def pribil(target_valid,predictions_valid,count):
    top_mining = predictions_valid.sort_values(ascending=False)
    target_choice = target_valid[top_mining.index][:count]
    income = (target_choice.sum() * revenue_per_barrel) - budget
    return income

## Расчёт прибыли и рисков 

In [36]:
def income_risks (target_valid, predictions_valid,  bootstrap_samples, explore):
    state = np.random.RandomState(12345)
    values = []
    
    for i in range(bootstrap_samples):
        target_subsample = target_valid.reset_index(drop=True).sample(explore, replace=True, random_state=state)
        probs_subsample = predictions_valid[target_subsample.index]
        rev = pribil(target_subsample, probs_subsample,mining)
        values.append(rev)

    values = pd.Series(values)

    lower = values.quantile(0.05) 
    upper = values.quantile(0.95) 
    mean = values.mean()
    risks = len(values[values < 0])/len(values)*100
    
    print("Средняя выручка:", mean)
    print("95%-квантиль по выручке:", upper)
    print("5%-квантиль по выручке:", lower)
    print("Вероятность убытков:",risks,"%")
    
    #values.hist()
    

In [37]:
print('Для первого региона при случайном выборе 500 скважин получаются следующие показатели:')
income_risks(target_valid_1, predictions_valid_1,  bootstrap_number, explore)

Для первого региона при случайном выборе 500 скважин получаются следующие показатели:
Средняя выручка: 425938526.91059244
95%-квантиль по выручке: 881341454.3602811
5%-квантиль по выручке: -31803114.346116535
Вероятность убытков: 6.0 %


In [38]:
print('Для второго региона при случайном выборе 500 скважин получаются следующие показатели:')
income_risks(target_valid_2, predictions_valid_2,  bootstrap_number, explore)

Для второго региона при случайном выборе 500 скважин получаются следующие показатели:
Средняя выручка: 515222773.4432899
95%-квантиль по выручке: 861521464.1012802
5%-квантиль по выручке: 150785740.64118025
Вероятность убытков: 1.0 %


In [39]:
print('Для третьего региона при случайном выборе 500 скважин получаются следующие показатели:')
income_risks(target_valid_3, predictions_valid_3,  bootstrap_number, explore)

Для третьего региона при случайном выборе 500 скважин получаются следующие показатели:
Средняя выручка: 435008362.7827556
95%-квантиль по выручке: 897954688.9016576
5%-квантиль по выручке: -43448491.32250163
Вероятность убытков: 6.4 %


### Вывод:
Наиболее лучшим регионом по добыче является - второй. При этом только в данном регионе вероятность убытков лежит в допустимом значении - 1% (Максимальное - 2.5%). Максимальная прибыль меньше, но не сильно отличается от 2х дргуих регионов. А вот средняя выручка больше, чем у первого и третьего региона.

## Вывод

1. Был изучен полученный файл. Пропусков и дубликатов не обнаружено;
2. Удален столбец `id` так как в дальнейшем анализе не участвует;
3. Данные были разбиты на 2 выборки:  обучающая и валидационная в соотношении 75:25;
4. Осуществленно масштабирование признаков;
5. Обучены модели для каждого региона. Наилучшая модель получилась у второго региона с наибольшой точностью предсказания;
6. Определен минимальный объем сырья для безубыточной работы. Данный объем превышает средние значения сырья в регионах, что говорит о возможных убытках
7. С помощью Bootstrap посчитаны прибиль и риски. Наилучшем регионом оказался второй. Вероятность убытков составила всего 1%, а максимальная выручка не сильно отличается от других регионов, при этом средняя вырчка выше чем у первого и третьего региона.